# Threat Intelligence Data Collection for LLM-TIKG

This notebook implements comprehensive data collection for Threat Intelligence Knowledge Graph (TIKG) construction using Large Language Models, following the LLM-TIKG paper methodology.

## Overview

The notebook covers:
1. **Web scraping setup** for major threat intelligence platforms
2. **Data cleaning and preprocessing** functions
3. **Text extraction** with paragraph structure preservation
4. **Data validation** and quality checks
5. **Structured storage** for downstream processing

## Target Platforms
- Symantec Security Center
- Fortinet Threat Intelligence
- TrendMicro Security Blog
- CISA Advisories

## Methodology
Following the LLM-TIKG paper, we focus on collecting high-quality threat intelligence data that preserves:
- Contextual relationships between entities
- Temporal information
- Technical details and indicators
- Structured narrative flow


## 1. Environment Setup and Imports


In [1]:
# Core libraries
import requests
from bs4 import BeautifulSoup, Tag
from typing import cast
import pandas as pd
import numpy as np
import json
import re
import time
import random
import logging
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from urllib.parse import urljoin, urlparse
import warnings

# NLP libraries
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag

# Web scraping utilities
from fake_useragent import UserAgent
import lxml

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
warnings.filterwarnings('ignore')

# Download required NLTK data
nltk_downloads = ['punkt', 'stopwords', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words']
for item in nltk_downloads:
    try:
        nltk.data.find(f'tokenizers/{item}')
    except LookupError:
        nltk.download(item, quiet=True)

print("✅ All required libraries imported successfully")


✅ All required libraries imported successfully


## 2. Configuration and Setup

Setting up logging, data directories, and scraping configurations following security best practices.


In [2]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('../data/raw/scraping.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Data directories
DATA_DIR = Path('../data')
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'

# Create directories if they don't exist
for dir_path in [RAW_DATA_DIR, PROCESSED_DATA_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Scraping configuration
SCRAPING_CONFIG = {
    'rate_limit': 2.0,  # seconds between requests
    'timeout': 30,      # request timeout
    'max_retries': 3,   # maximum retry attempts
    'batch_size': 50,   # articles per batch
    'headers': {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }
}

# Initialize user agent rotator
ua = UserAgent()

print("✅ Configuration setup complete")
print(f"📁 Raw data directory: {RAW_DATA_DIR}")
print(f"📁 Processed data directory: {PROCESSED_DATA_DIR}")


✅ Configuration setup complete
📁 Raw data directory: ..\data\raw
📁 Processed data directory: ..\data\processed


## 3. Core Web Scraping Infrastructure

Building robust web scraping infrastructure with rate limiting, error handling, and retry mechanisms.


In [3]:
class ThreatIntelligenceScraper:
    """
    A robust web scraper for threat intelligence data collection.
    Implements rate limiting, error handling, and data validation.
    """
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.session = requests.Session()
        self.session.headers.update(config['headers'])
        self.last_request_time = 0
        self.ua = UserAgent()
        
    def _rate_limit(self):
        """Implement rate limiting between requests with random delay."""
        elapsed = time.time() - self.last_request_time
        base_sleep_time = self.config['rate_limit'] - elapsed
        
        # Add random delay if configured
        if 'random_delay' in self.config:
            min_delay, max_delay = self.config['random_delay']
            random_delay = min_delay + (max_delay - min_delay) * np.random.random()
            base_sleep_time += random_delay
        
        if base_sleep_time > 0:
            time.sleep(base_sleep_time)
        self.last_request_time = time.time()
    
    def _get_page(self, url: str, max_retries: int = 0) -> Optional[BeautifulSoup]:
        """Fetch a web page with enhanced retry logic and error handling."""
        if max_retries == 0:
            max_retries = self.config['max_retries'] or 1
        
        for attempt in range(max_retries + 1):
            try:
                self._rate_limit()
                
                # Rotate user agent and add additional headers
                self.session.headers.update({
                    'User-Agent': self.ua.random,
                    'Cache-Control': 'no-cache',
                    'Pragma': 'no-cache',
                    'DNT': '1'  # Do Not Track
                })
                
                # Log request attempt
                logger.info(f"Attempting to fetch {url} (attempt {attempt + 1}/{max_retries + 1})")
                
                response = self.session.get(
                    url, 
                    timeout=self.config['timeout'],
                    allow_redirects=True,
                    verify=True  # Enforce SSL verification
                )
                
                # Check response before processing
                if response.status_code == 403:
                    logger.warning(f"Access forbidden (403) for {url}")
                    time.sleep(5 ** attempt)  # Longer delay for 403s
                    continue
                    
                response.raise_for_status()
                
                # Check content type
                content_type = response.headers.get('content-type', '').lower()
                if 'text/html' not in content_type:
                    logger.warning(f"Unexpected content type: {content_type}")
                
                # Parse with BeautifulSoup with error handling
                try:
                    soup = BeautifulSoup(response.content, 'lxml')
                    
                    # Basic validation of parsed content
                    if not soup.find('body'):
                        logger.warning("No <body> tag found in response")
                        if attempt < max_retries:
                            continue
                    
                    logger.info(f"Successfully scraped: {url}")
                    return soup
                    
                except Exception as parse_error:
                    logger.error(f"Failed to parse HTML: {str(parse_error)}")
                    if attempt < max_retries:
                        continue
                    return None
                
            except requests.exceptions.RequestException as e:
                error_msg = str(e)
                logger.warning(f"Attempt {attempt + 1} failed for {url}: {error_msg}")
                
                # Specific handling for common errors
                if "SSLError" in error_msg:
                    logger.error("SSL verification failed")
                elif "Timeout" in error_msg:
                    logger.error("Request timed out")
                elif "ConnectionError" in error_msg:
                    logger.error("Failed to establish connection")
                
                if attempt < max_retries:
                    # Exponential backoff with jitter
                    delay = (2 ** attempt) + (random.random() * 2)
                    time.sleep(delay)
                else:
                    logger.error(f"Failed to scrape {url} after {max_retries + 1} attempts")
                    return None
    
    def extract_text_content(self, soup: BeautifulSoup, selectors: Dict[str, str]) -> Dict[str, Any]:
        """Extract structured text content from HTML using CSS selectors."""
        content = {}
        
        for field, selector in selectors.items():
            try:
                elements = soup.select(selector)
                if elements:
                    if field in ['title', 'date', 'author']:
                        content[field] = elements[0].get_text(strip=True)
                    else:
                        # Preserve paragraph structure for main content
                        content[field] = [elem.get_text(strip=True) for elem in elements if elem.get_text(strip=True)]
                else:
                    content[field] = None
            except Exception as e:
                logger.warning(f"Failed to extract {field}: {str(e)}")
                content[field] = None
        
        return content
    
    def validate_content(self, content: Dict[str, Any]) -> bool:
        """Validate extracted content quality."""
        # Check if essential fields are present
        required_fields = ['title', 'content']
        for field in required_fields:
            if not content.get(field):
                return False
        
        # Check content length (avoid very short articles)
        if isinstance(content['content'], list):
            total_length = sum(len(p) for p in content['content'])
        else:
            total_length = len(content['content']) if content['content'] else 0
        
        return total_length > 50  # Reduced from 100 to 50 for testing

# Initialize scraper
scraper = ThreatIntelligenceScraper(SCRAPING_CONFIG)
print("✅ Threat Intelligence Scraper initialized")

✅ Threat Intelligence Scraper initialized


## 4. Platform-Specific Scrapers

Implementing specialized scrapers for each threat intelligence platform with their unique HTML structures.


In [4]:
# Platform-specific scrapers
class CISAScraper:
    """Enhanced scraper for CISA (Cybersecurity and Infrastructure Security Agency) advisories."""
    
    def __init__(self, base_scraper: ThreatIntelligenceScraper):
        self.scraper = base_scraper
        self.base_url = "https://www.cisa.gov"
        self.advisories_url = "https://www.cisa.gov/news-events/cybersecurity-advisories"
        
        self.selectors = {
            'title': '.title, .advisory-title, .usa-accordion__heading, h1.usa-prose',
            'date': '.published-date, .date-published, time, .usa-prose time',
            'content': '.usa-prose p, .field--type-text-with-summary p, .field--name-body p, article p',
            'severity': '.severity, .risk-level, .tlp-label',
            'advisory_id': '.advisory-id, .alert-code, .reference-number'
        }
    
    def get_article_links(self, max_pages: int = 5) -> List[str]:
        """Extract article links from CISA advisories with pagination support."""
        links = []
        
        try:
            for page in range(1, max_pages + 1):
                page_url = f"{self.advisories_url}?page={page}" if page > 1 else self.advisories_url
                print(f"   Fetching CISA page {page}/{max_pages}: {page_url}")
                
                soup = self.scraper._get_page(page_url)
                if not soup:
                    print(f"   ⚠️ Failed to fetch page {page}")
                    break
                
                # Try multiple selector patterns
                selectors = [
                    'a[href*="/advisory/"]',
                    'a[href*="/alert/"]',
                    '.views-row a',
                    '.usa-collection__item a',
                    'article a'
                ]
                
                page_links_count = 0
                for selector in selectors:
                    article_links = soup.select(selector)
                    for link in article_links:
                        href = link.get('href')
                        if href and ('/advisory/' in href or '/alert/' in href):
                            full_url = urljoin(self.base_url, href)
                            if full_url not in links:
                                links.append(full_url)
                                page_links_count += 1
                
                print(f"      Found {page_links_count} new articles on page {page}")
                if page_links_count == 0:  # No more articles found
                    break
                            
        except Exception as e:
            print(f"   ❌ Error getting CISA article links: {str(e)}")
            logger.error(f"CISA link extraction failed: {str(e)}")
        
        print(f"   Total CISA articles found: {len(links)}")
        return links

print("✅ CISA scrapers initialized")


✅ CISA scrapers initialized


In [11]:
class FortinetScraper:
    """Corrected scraper for Fortinet Threat Research blog using URL path extensions."""
    
    def __init__(self, base_scraper):
        self.scraper = base_scraper
        self.base_url = "https://www.fortinet.com"
        self.blog_url = "https://www.fortinet.com/blog/threat-research"
        self.load_more_url = "https://www.fortinet.com/content/fortinet-blog/us/en/threat-research/jcr:content/root/bloglist"
        
        self.selectors = {
            'title': 'h1, h2, .title, .headline',
            'date': '.date, .time, .published, time',
            'content': 'p, .content, .text, .body',
            'author': '.author, .byline, .writer',
            'category': '.category, .tags, .topic'
        }
    
    def get_article_links(self, max_pages: int = 30) -> List[str]:
        """Extract article links from Fortinet blog using URL path extensions for pagination."""
        links = []
        
        try:
            # First get the initial page (page 0)
            print(f"   Fetching initial page: {self.blog_url}")
            soup = self.scraper._get_page(self.blog_url)
            if soup:
                # Find article links on the first page
                article_links = soup.find_all('a', href=True)
                for link in article_links:
                    if isinstance(link, Tag):
                        href = link.get('href')
                        if isinstance(href, str):
                            href_lower = href.lower()
                            if '/blog/' in href_lower and 'threat' in href_lower:
                                if href.startswith('/'):
                                    full_url = urljoin(self.base_url, href)
                                else:
                                    full_url = href
                                if full_url not in links and 'threat-research' in full_url:
                                    links.append(full_url)
                
                print(f"      Found {len(links)} articles on initial page")
                
                # Now use the URL path extension approach for pagination
                for page in range(1, max_pages + 1):
                    print(f"   Loading page {page}/{max_pages}")
                    
                    # Use URL path extension approach
                    page_url = f"{self.load_more_url}.{page}"
                    
                    try:
                        self.scraper._rate_limit()
                        response = self.scraper.session.get(
                            page_url,
                            timeout=self.scraper.config['timeout']
                        )
                        response.raise_for_status()
                        
                        # Parse HTML content from response
                        soup = BeautifulSoup(response.text, 'lxml')
                        article_links = soup.find_all('a', href=True)
                        
                        new_links = 0
                        for link in article_links:
                            if isinstance(link, Tag):
                                href = link.get('href')
                                if isinstance(href, str):
                                    href_lower = href.lower()
                                    if '/blog/' in href_lower and 'threat' in href_lower:
                                        if href.startswith('/'):
                                            full_url = urljoin(self.base_url, href)
                                        else:
                                            full_url = href
                                        if full_url not in links and 'threat-research' in full_url:
                                            links.append(full_url)
                                            new_links += 1
                        
                        print(f"      Found {new_links} new articles on page {page}")
                        
                        if new_links == 0:
                            print("      No more new articles found")
                            break
                        
                        # Stop if we've collected enough articles
                        if len(links) >= 300:  # Safety limit
                            print(f"      Reached article limit (300)")
                            break
                            
                    except Exception as e:
                        print(f"      Error loading page {page}: {str(e)}")
                        break
                            
        except Exception as e:
            print(f"   ❌ Error getting Fortinet article links: {str(e)}")
            logger.error(f"Fortinet link extraction failed: {str(e)}")
        
        print(f"   Total Fortinet articles found: {len(links)}")
        return links
    
    def scrape_article(self, url: str) -> Optional[Dict[str, Any]]:
        """Scrape a single Fortinet blog post with improved validation."""
        print(f"\n🔍 Scraping Fortinet article: {url}")
        
        soup = self.scraper._get_page(url)
        if not soup:
            print("   ❌ Failed to fetch article")
            return None
        
        content = self.scraper.extract_text_content(soup, self.selectors)
        content['url'] = url
        content['source'] = 'Fortinet'
        content['scraped_at'] = datetime.now().isoformat()
        
        if self.scraper.validate_content(content):
            print("   ✅ Successfully scraped article")
            print(f"      Title: {content.get('title', 'N/A')[:100]}")
            print(f"      Content length: {len(str(content.get('content', '')))}")
            return content
            
        print("   ❌ Content validation failed")
        return None

print("✅  Fortinet scraper initialized")


✅  Fortinet scraper initialized


In [6]:
class SymantecScraper:
    """Enhanced scraper for Symantec Security Center blog posts with pagination."""
    
    def __init__(self, base_scraper: ThreatIntelligenceScraper):
        self.scraper = base_scraper
        self.base_url = "https://symantec-enterprise-blogs.security.com"
        self.blog_url = "https://symantec-enterprise-blogs.security.com/blogs/threat-research"
        
        self.selectors = {
            'title': 'h1, .post-title, .entry-title',
            'date': '.date, .published, .post-date',
            'content': '.post-content p, .entry-content p, .content p',
            'author': '.author, .byline',
            'tags': '.tags, .categories'
        }
    
    def get_article_links(self, max_pages: int = 30) -> List[str]:
        """Extract article links from Symantec blog with pagination support."""
        links = []
        
        try:
            for page in range(1, max_pages + 1):
                # Try different pagination URL patterns
                page_urls = [
                    f"{self.blog_url}?page={page-1}" if page > 1 else self.blog_url,
                    f"{self.blog_url}/page/{page}" if page > 1 else self.blog_url,
                    f"{self.blog_url}?p={page}" if page > 1 else self.blog_url
                ]
                
                page_found = False
                for page_url in page_urls:
                    print(f"   Trying page {page}/{max_pages}: {page_url}")
                    
                    soup = self.scraper._get_page(page_url)
                    if not soup:
                        continue
                    
                    # Try multiple selector patterns
                    selectors = [
                        'article a',
                        '.post-title a',
                        '.entry-title a',
                        'a[href*="/blogs/threat-research/"]'
                    ]
                    
                    page_links_count = 0
                    for selector in selectors:
                        article_links = soup.select(selector)
                        for link in article_links:
                            if isinstance(link, Tag):
                                href = link.get('href')
                                if isinstance(href, str) and ('/blogs/' in href or '/threat-research/' in href):
                                    if href.startswith('/'):
                                        full_url = urljoin(self.base_url, href)
                                    else:
                                        full_url = href
                                    if full_url not in links and 'threat-research' in full_url:
                                        links.append(full_url)
                                        page_links_count += 1
                    
                    if page_links_count > 0:
                        print(f"      Found {page_links_count} new articles")
                        page_found = True
                        break  # Found articles on this page, no need to try other URL patterns
                    
                if not page_found:
                    print(f"      No articles found on page {page}, stopping pagination")
                    break  # No articles found with any URL pattern, likely reached the end
                
                # Stop if we've collected enough articles
                if len(links) >= 300:  # Safety limit
                    print(f"      Reached article limit (300)")
                    break
                            
        except Exception as e:
            print(f"   ❌ Error getting Symantec article links: {str(e)}")
            logger.error(f"Symantec link extraction failed: {str(e)}")
        
        print(f"   Total Symantec articles found: {len(links)}")
        return links
    
    def scrape_article(self, url: str) -> Optional[Dict[str, Any]]:
        """Scrape a single Symantec blog post with improved validation."""
        print(f"\n🔍 Scraping Symantec article: {url}")
        
        soup = self.scraper._get_page(url)
        if not soup:
            print("   ❌ Failed to fetch article")
            return None
        
        content = self.scraper.extract_text_content(soup, self.selectors)
        content['url'] = url
        content['source'] = 'Symantec'
        content['scraped_at'] = datetime.now().isoformat()
        
        if self.scraper.validate_content(content):
            print("   ✅ Successfully scraped article")
            print(f"      Title: {content.get('title', 'N/A')[:100]}")
            print(f"      Content length: {len(str(content.get('content', '')))}")
            return content
            
        print("   ❌ Content validation failed")
        return None

print("✅ Symantec scrapers initialized")

✅ Symantec scrapers initialized


## 5. Text Processing and Preprocessing

Advanced text processing functions to clean, normalize, and structure the collected threat intelligence data.


In [7]:
class ThreatIntelligenceProcessor:
    """
    Advanced text processor for threat intelligence data.
    Handles cleaning, normalization, and structure preservation.
    """
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        
        # Threat intelligence specific patterns
        self.patterns = {
            'ip_address': re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b'),
            'domain': re.compile(r'\b[a-zA-Z0-9]([a-zA-Z0-9\-]{0,61}[a-zA-Z0-9])?\.[a-zA-Z]{2,}\b'),
            'hash_md5': re.compile(r'\b[a-fA-F0-9]{32}\b'),
            'hash_sha1': re.compile(r'\b[a-fA-F0-9]{40}\b'),
            'hash_sha256': re.compile(r'\b[a-fA-F0-9]{64}\b'),
            'cve': re.compile(r'CVE-\d{4}-\d{4,7}', re.IGNORECASE),
            'email': re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'),
            'url': re.compile(r'https?://(?:[-\w.])+(?:[:\d]+)?(?:/(?:[\w/_.])*(?:\?(?:[\w&=%.])*)?(?:#(?:\w)*)?)?'),
        }
        
        # Threat keywords for relevance scoring
        self.threat_keywords = [
            'apt', 'advanced persistent threat', 'ransomware', 'trojan', 'backdoor',
            'botnet', 'malware', 'phishing', 'spear phishing', 'zero-day',
            'exploit', 'vulnerability', 'attack', 'campaign', 'threat actor'
        ]
    
    def clean_text(self, text: str) -> str:
        """Clean and normalize text while preserving important technical details."""
        if not text:
            return ""
        
        # Remove HTML entities and extra whitespace
        text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        
        # Preserve important punctuation in technical contexts
        text = re.sub(r'([.!?])([A-Z])', r'\1 \2', text)
        
        return text.strip()
    
    def extract_indicators(self, text: str) -> Dict[str, List[str]]:
        """Extract cybersecurity indicators from text."""
        indicators = {}
        
        for indicator_type, pattern in self.patterns.items():
            matches = pattern.findall(text)
            indicators[indicator_type] = list(set(matches))  # Remove duplicates
        
        return indicators
    
    def calculate_threat_relevance_score(self, text: str) -> float:
        """Calculate how relevant the text is to threat intelligence."""
        if not text:
            return 0.0
        
        text_lower = text.lower()
        score = 0.0
        
        # Count threat-related keywords
        for keyword in self.threat_keywords:
            count = text_lower.count(keyword)
            score += count * 0.1
        
        # Bonus for technical indicators
        indicators = self.extract_indicators(text)
        for indicator_type, matches in indicators.items():
            if matches:
                score += len(matches) * 0.2
        
        # Normalize by text length
        text_length = len(text.split())
        if text_length > 0:
            score = score / (text_length / 100)  # Per 100 words
        
        return min(score, 10.0)  # Cap at 10.0
    
    def process_article(self, article_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process a complete article with all preprocessing steps."""
        processed = article_data.copy()
        
        # Clean title
        if processed.get('title'):
            processed['title'] = self.clean_text(processed['title'])
        
        # Process content
        if processed.get('content'):
            if isinstance(processed['content'], list):
                # Preserve paragraph structure
                cleaned_paragraphs = []
                for paragraph in processed['content']:
                    cleaned = self.clean_text(paragraph)
                    if len(cleaned) > 10:  # Filter out very short paragraphs
                        cleaned_paragraphs.append(cleaned)
                
                processed['paragraphs'] = cleaned_paragraphs
                processed['full_text'] = ' '.join(cleaned_paragraphs)
            else:
                # Single string content
                clean_content = self.clean_text(processed['content'])
                processed['full_text'] = clean_content
                processed['paragraphs'] = [clean_content]
        
        # Extract indicators and calculate relevance
        if processed.get('full_text'):
            processed['indicators'] = self.extract_indicators(processed['full_text'])
            processed['threat_relevance_score'] = self.calculate_threat_relevance_score(processed['full_text'])
        
        # Add processing metadata
        processed['processed_at'] = datetime.now().isoformat()
        processed['processing_version'] = '1.0'
        
        return processed

# Initialize processor
processor = ThreatIntelligenceProcessor()
print("✅ Threat Intelligence Processor initialized")
print(f"🔍 Monitoring {len(processor.patterns)} indicator types")
print(f"📝 Tracking {len(processor.threat_keywords)} threat keywords")


✅ Threat Intelligence Processor initialized
🔍 Monitoring 8 indicator types
📝 Tracking 15 threat keywords


## 6. Data Validation and Quality Checks

Comprehensive data validation system to ensure high-quality threat intelligence collection.


In [8]:
class DataValidator:
    """
    Comprehensive data validation for threat intelligence articles.
    Ensures data quality and consistency for downstream processing.
    """
    
    def __init__(self):
        self.validation_rules = {
            'min_content_length': 100,
            'max_content_length': 50000,
            'min_title_length': 5,
            'max_title_length': 200,
            'min_threat_score': 0.1,
            'required_fields': ['title', 'content', 'source', 'url']
        }
    
    def validate_structure(self, article: Dict[str, Any]) -> Tuple[bool, List[str]]:
        """Validate article structure and required fields."""
        errors = []
        
        # Check required fields
        for field in self.validation_rules['required_fields']:
            if not article.get(field):
                errors.append(f"Missing required field: {field}")
        
        # Validate title
        title = article.get('title', '')
        if len(title) < self.validation_rules['min_title_length']:
            errors.append(f"Title too short: {len(title)} chars")
        elif len(title) > self.validation_rules['max_title_length']:
            errors.append(f"Title too long: {len(title)} chars")
        
        # Validate content length
        content_length = 0
        if article.get('full_text'):
            content_length = len(article['full_text'])
        elif article.get('content'):
            if isinstance(article['content'], list):
                content_length = sum(len(p) for p in article['content'])
            else:
                content_length = len(article['content'])
        
        if content_length < self.validation_rules['min_content_length']:
            errors.append(f"Content too short: {content_length} chars")
        elif content_length > self.validation_rules['max_content_length']:
            errors.append(f"Content too long: {content_length} chars")
        
        return len(errors) == 0, errors
    
    def generate_quality_report(self, article: Dict[str, Any]) -> Dict[str, Any]:
        """Generate comprehensive quality report for an article."""
        report = {
            'article_id': article.get('url', 'unknown'),
            'source': article.get('source', 'unknown'),
            'validation_timestamp': datetime.now().isoformat()
        }
        
        # Structure validation
        structure_valid, structure_errors = self.validate_structure(article)
        report['structure'] = {
            'valid': structure_valid,
            'errors': structure_errors
        }
        
        # Calculate overall quality score
        quality_score = 0
        if structure_valid:
            quality_score += 4
        
        # Check threat relevance
        threat_score = article.get('threat_relevance_score', 0.0)
        if threat_score >= self.validation_rules['min_threat_score']:
            quality_score += 3
        
        # Check for technical indicators
        indicators = article.get('indicators', {})
        total_indicators = sum(len(inds) for inds in indicators.values())
        if total_indicators > 0:
            quality_score += 3
        
        report['overall'] = {
            'score': quality_score,
            'max_score': 10,
            'grade': self._calculate_grade(quality_score, 10),
            'approved': quality_score >= 7
        }
        
        return report
    
    def _calculate_grade(self, score: float, max_score: float) -> str:
        """Calculate letter grade based on score."""
        percentage = (score / max_score) * 100
        if percentage >= 90:
            return 'A'
        elif percentage >= 80:
            return 'B'
        elif percentage >= 70:
            return 'C'
        elif percentage >= 60:
            return 'D'
        else:
            return 'F'

# Initialize validator
validator = DataValidator()
print("✅ Data Validator initialized")
print(f"📋 Validation rules: {len(validator.validation_rules)} criteria")


✅ Data Validator initialized
📋 Validation rules: 6 criteria


In [9]:
# Export functions for data processing
def export_for_llm_training(articles: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Export data in formats suitable for LLM training and knowledge graph construction."""
    
    if not articles:
        print("❌ No articles to export")
        return {}
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. Training text format (for LLM fine-tuning)
    training_texts = []
    for article in articles:
        text_entry = {
            'id': f"{article['source']}_{hash(article['url']) % 100000}",
            'source': article['source'],
            'title': article['title'],
            'text': article['full_text'],
            'metadata': {
                'url': article['url'],
                'scraped_at': article['scraped_at'],
                'indicators': article.get('indicators', {}),
                'threat_score': article.get('threat_relevance_score', 0)
            }
        }
        training_texts.append(text_entry)
    
    training_file = PROCESSED_DATA_DIR / f'llm_training_data_{timestamp}.json'
    with open(training_file, 'w', encoding='utf-8') as f:
        json.dump(training_texts, f, indent=2, ensure_ascii=False)
    
    # 2. Entity-Relationship format (for knowledge graph)
    entities_and_relations = {
        'entities': [],
        'relations': [],
        'documents': []
    }
    
    for idx, article in enumerate(articles):
        doc_id = f"doc_{idx}"
        
        # Document node
        entities_and_relations['documents'].append({
            'id': doc_id,
            'title': article['title'],
            'source': article['source'],
            'url': article['url'],
            'threat_score': article.get('threat_relevance_score', 0)
        })
        
        # Extract entities from indicators
        indicators = article.get('indicators', {})
        for indicator_type, values in indicators.items():
            for value in values:
                entity_id = f"{indicator_type}_{hash(value) % 100000}"
                
                # Entity
                entities_and_relations['entities'].append({
                    'id': entity_id,
                    'type': indicator_type,
                    'value': value
                })
                
                # Relation
                entities_and_relations['relations'].append({
                    'source': doc_id,
                    'target': entity_id,
                    'relation': 'mentions',
                    'type': indicator_type
                })
    
    # Remove duplicate entities
    seen_entities = set()
    unique_entities = []
    for entity in entities_and_relations['entities']:
        entity_key = (entity['type'], entity['value'])
        if entity_key not in seen_entities:
            seen_entities.add(entity_key)
            unique_entities.append(entity)
    entities_and_relations['entities'] = unique_entities
    
    kg_file = PROCESSED_DATA_DIR / f'knowledge_graph_data_{timestamp}.json'
    with open(kg_file, 'w', encoding='utf-8') as f:
        json.dump(entities_and_relations, f, indent=2, ensure_ascii=False)
    
    # 3. JSONL format (for streaming/batch processing)
    jsonl_file = PROCESSED_DATA_DIR / f'threat_intelligence_{timestamp}.jsonl'
    with open(jsonl_file, 'w', encoding='utf-8') as f:
        for article in articles:
            simplified_article = {
                'title': article['title'],
                'content': article['full_text'],
                'source': article['source'],
                'indicators': article.get('indicators', {}),
                'threat_score': article.get('threat_relevance_score', 0)
            }
            f.write(json.dumps(simplified_article, ensure_ascii=False) + '\n')
    
    # 4. Summary statistics
    export_summary = {
        'export_timestamp': datetime.now().isoformat(),
        'total_articles': len(articles),
        'sources': list(set(article['source'] for article in articles)),
        'total_entities': len(entities_and_relations['entities']),
        'total_relations': len(entities_and_relations['relations']),
        'files_created': {
            'llm_training': str(training_file),
            'knowledge_graph': str(kg_file),
            'jsonl_format': str(jsonl_file)
        },
        'statistics': {
            'avg_threat_score': sum(article.get('threat_relevance_score', 0) for article in articles) / len(articles),
            'total_technical_indicators': sum(sum(len(inds) for inds in article.get('indicators', {}).values()) for article in articles)
        }
    }
    
    summary_file = PROCESSED_DATA_DIR / f'export_summary_{timestamp}.json'
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(export_summary, f, indent=2, ensure_ascii=False)
    
    print("\n✅ Export completed successfully!")
    print("="*60)
    print(f"📊 Exported {len(articles)} articles")
    print(f"🎯 {len(entities_and_relations['entities'])} unique entities")
    print(f"🔗 {len(entities_and_relations['relations'])} relations")
    print(f"📈 Avg threat score: {export_summary['statistics']['avg_threat_score']:.2f}")
    
    print("\n📁 Files created:")
    for file_type, file_path in export_summary['files_created'].items():
        print(f"  {file_type}: {Path(file_path).name}")
    
    return export_summary

print("✅ Export functions initialized")


✅ Export functions initialized


## 7. Data Collection Execution

Execute the data collection process with monitoring and comprehensive error handling.


In [12]:
# Enhanced configuration for multi-source data collection
COLLECTION_CONFIG = {
    'max_articles_per_platform': 300,    # Maximum articles to collect per platform
    'enable_progress_tracking': True,
    'save_intermediate_results': True,
    'rate_limit_multiplier': 3.0,      # Conservative rate limiting
    'max_retries': 5,                  # More retries for reliability
    'timeout': 45,                     # Longer timeout for slow sites
    'random_delay': (1, 3)             # Random delay between requests
}

print("🌐 Multi-Platform Threat Intelligence Collection")
print("="*60)

# Initialize scrapers with enhanced configuration
live_scraping_config = SCRAPING_CONFIG.copy()
live_scraping_config.update({
    'rate_limit': SCRAPING_CONFIG['rate_limit'] * COLLECTION_CONFIG['rate_limit_multiplier'],
    'max_retries': COLLECTION_CONFIG['max_retries'],
    'timeout': COLLECTION_CONFIG['timeout'],
    'random_delay': COLLECTION_CONFIG['random_delay']
})

live_scraper = ThreatIntelligenceScraper(live_scraping_config)

# Initialize all platform scrapers with our improved versions
scrapers = {
    'CISA': CISAScraper(live_scraper),
    'Fortinet': FortinetScraper(live_scraper),
    'Symantec': SymantecScraper(live_scraper)
}

print("\n⚙️ Scraping Configuration:")
print(f"  Rate Limit: {live_scraping_config['rate_limit']} seconds")
print(f"  Max Retries: {live_scraping_config['max_retries']}")
print(f"  Timeout: {live_scraping_config['timeout']} seconds")
print(f"  Sources: {', '.join(scrapers.keys())}\n")

def collect_threat_intelligence_data():
    """Enhanced function to collect and process threat intelligence data from multiple sources."""
    
    start_time = datetime.now()
    logger.info(f"Collection started at {start_time}")
    
    all_collected = []
    sources_status = {}
    
    try:
        # Process each source
        for source_name, source_scraper in scrapers.items():
            print(f"\n📡 Collecting from {source_name}...")
            
            try:
                # Get article links with increased max_pages
                links = source_scraper.get_article_links(max_pages=30)  # Try up to 30 pages
                print(f"   Found {len(links)} potential articles")
                
                source_collected = 0
                source_attempted = min(len(links), COLLECTION_CONFIG['max_articles_per_platform'])
                
                # Process articles (limited by max_articles_per_platform)
                for i, url in enumerate(links[:COLLECTION_CONFIG['max_articles_per_platform']], 1):
                    print(f"   Processing article {i}/{source_attempted}: {url[:60]}...")
                    
                    try:
                        # Scrape and process article
                        article = source_scraper.scrape_article(url)
                        
                        if article:
                            processed = processor.process_article(article)
                            quality_report = validator.generate_quality_report(processed)
                            processed['quality_report'] = quality_report
                            
                            if quality_report['overall']['approved']:
                                all_collected.append(processed)
                                source_collected += 1
                                print(f"   ✅ Approved (Grade: {quality_report['overall']['grade']})")
                                
                                # Show extracted indicators
                                indicators = processed.get('indicators', {})
                                total_iocs = sum(len(inds) for inds in indicators.values())
                                if total_iocs > 0:
                                    print(f"   🔍 Extracted {total_iocs} technical indicators")
                                    for ioc_type, ioc_list in indicators.items():
                                        if ioc_list:
                                            print(f"      {ioc_type}: {len(ioc_list)} found")
                            else:
                                print(f"   ❌ Rejected (Grade: {quality_report['overall']['grade']})")
                        else:
                            print(f"   ❌ Failed to scrape")
                            
                    except Exception as e:
                        print(f"   ❌ Error processing article: {str(e)[:100]}")
                        continue
                    
                    # Rate limiting between articles
                    time.sleep(live_scraping_config['rate_limit'])
                
                # Record source statistics
                sources_status[source_name] = {
                    'attempted': source_attempted,
                    'collected': source_collected,
                    'success_rate': source_collected / source_attempted if source_attempted > 0 else 0
                }
                
            except Exception as e:
                print(f"   ❌ Source failed: {str(e)}")
                sources_status[source_name] = {'error': str(e)}
        
        # Save collected data
        if all_collected:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            output_file = RAW_DATA_DIR / f'threat_intelligence_multi_source_{timestamp}.json'
            
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(all_collected, f, indent=2, ensure_ascii=False)
            
            print(f"\n💾 Saved {len(all_collected)} articles from multiple sources")
            
            # Export for LLM training
            export_summary = export_for_llm_training(all_collected)
        else:
            print("\n⚠️  No articles were successfully collected")
        
        # Display summary
        end_time = datetime.now()
        duration = end_time - start_time
        
        print("\n" + "="*70)
        print("📊 COLLECTION SUMMARY")
        print("="*70)
        print(f"⏱️  Duration: {duration}")
        
        # Source-specific statistics
        for source, status in sources_status.items():
            if 'error' in status:
                print(f"{source}: ❌ {status['error']}")
            else:
                success_rate = status['success_rate'] * 100
                print(f"{source}: {status['collected']}/{status['attempted']} ({success_rate:.1f}%)")
        
        if all_collected:
            print(f"\n📄 Total Articles Collected: {len(all_collected)}")
            
            # Calculate overall statistics
            avg_threat_score = sum(article.get('threat_relevance_score', 0) for article in all_collected) / len(all_collected)
            print(f"🎯 Average Threat Score: {avg_threat_score:.2f}")
            
            # Show total indicators
            total_indicators = sum(
                sum(len(inds) for inds in article.get('indicators', {}).values())
                for article in all_collected
            )
            print(f"🔍 Total Technical Indicators: {total_indicators}")
            
            # Show sources distribution
            sources_count = {}
            for article in all_collected:
                source = article.get('source', 'Unknown')
                sources_count[source] = sources_count.get(source, 0) + 1
            print("\n📊 Articles per source:")
            for source, count in sources_count.items():
                print(f"   {source}: {count}")
        
        print("\n🎉 Data collection completed successfully!")
        return all_collected
        
    except Exception as e:
        logger.error(f"Collection failed: {str(e)}")
        print(f"❌ Collection failed: {str(e)}")
        raise

# Execute collection
collected_data = collect_threat_intelligence_data()


2025-07-20 13:37:21,955 - INFO - Collection started at 2025-07-20 13:37:21.955543
2025-07-20 13:37:21,979 - INFO - Attempting to fetch https://www.cisa.gov/news-events/cybersecurity-advisories (attempt 1/6)


🌐 Multi-Platform Threat Intelligence Collection

⚙️ Scraping Configuration:
  Rate Limit: 6.0 seconds
  Max Retries: 5
  Timeout: 45 seconds
  Sources: CISA, Fortinet, Symantec


📡 Collecting from CISA...
   Fetching CISA page 1/30: https://www.cisa.gov/news-events/cybersecurity-advisories


2025-07-20 13:37:22,469 - INFO - Successfully scraped: https://www.cisa.gov/news-events/cybersecurity-advisories


      Found 0 new articles on page 1
   Total CISA articles found: 0
   Found 0 potential articles

📡 Collecting from Fortinet...
   Fetching initial page: https://www.fortinet.com/blog/threat-research


2025-07-20 13:37:29,627 - INFO - Attempting to fetch https://www.fortinet.com/blog/threat-research (attempt 1/6)
2025-07-20 13:37:29,913 - INFO - Successfully scraped: https://www.fortinet.com/blog/threat-research


...

   Trying page 1/30: https://symantec-enterprise-blogs.security.com/blogs/threat-research


2025-07-20 14:03:25,272 - INFO - Attempting to fetch https://symantec-enterprise-blogs.security.com/blogs/threat-research (attempt 1/6)
2025-07-20 14:03:25,434 - INFO - Successfully scraped: https://symantec-enterprise-blogs.security.com/blogs/threat-research


   Trying page 1/30: https://symantec-enterprise-blogs.security.com/blogs/threat-research


2025-07-20 14:03:33,761 - INFO - Attempting to fetch https://symantec-enterprise-blogs.security.com/blogs/threat-research (attempt 1/6)
2025-07-20 14:03:33,927 - INFO - Successfully scraped: https://symantec-enterprise-blogs.security.com/blogs/threat-research


      No articles found on page 1, stopping pagination
   Total Symantec articles found: 0
   Found 0 potential articles

💾 Saved 170 articles from multiple sources

✅ Export completed successfully!
📊 Exported 170 articles
🎯 2511 unique entities
🔗 2881 relations
📈 Avg threat score: 0.55

📁 Files created:
  llm_training: llm_training_data_20250720_140334.json
  knowledge_graph: knowledge_graph_data_20250720_140334.json
  jsonl_format: threat_intelligence_20250720_140334.jsonl

📊 COLLECTION SUMMARY
⏱️  Duration: 0:26:12.157983
CISA: 0/0 (0.0%)
Fortinet: 170/171 (99.4%)
Symantec: 0/0 (0.0%)

📄 Total Articles Collected: 170
🎯 Average Threat Score: 0.55
🔍 Total Technical Indicators: 2881

📊 Articles per source:
   Fortinet: 170

🎉 Data collection completed successfully!



## 8. Data Export and Format Conversion

Export the collected and validated data in various formats for downstream processing in the LLM-TIKG pipeline.


In [ ]:
def export_for_llm_training(articles: List[Dict[str, Any]]):
    """Export data in formats suitable for LLM training and knowledge graph construction."""
    
    if not articles:
        print("❌ No articles to export")
        return
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. Training text format (for LLM fine-tuning)
    training_texts = []
    for article in articles:
        text_entry = {
            'id': f"{article['source']}_{hash(article['url']) % 100000}",
            'source': article['source'],
            'title': article['title'],
            'text': article['full_text'],
            'metadata': {
                'url': article['url'],
                'scraped_at': article['scraped_at'],
                'indicators': article.get('indicators', {}),
                'threat_score': article.get('threat_relevance_score', 0)
            }
        }
        training_texts.append(text_entry)
    
    training_file = PROCESSED_DATA_DIR / f'llm_training_data_{timestamp}.json'
    with open(training_file, 'w', encoding='utf-8') as f:
        json.dump(training_texts, f, indent=2, ensure_ascii=False)
    
    # 2. Entity-Relationship format (for knowledge graph)
    entities_and_relations = {
        'entities': [],
        'relations': [],
        'documents': []
    }
    
    for idx, article in enumerate(articles):
        doc_id = f"doc_{idx}"
        
        # Document node
        entities_and_relations['documents'].append({
            'id': doc_id,
            'title': article['title'],
            'source': article['source'],
            'url': article['url'],
            'threat_score': article.get('threat_relevance_score', 0)
        })
        
        # Extract entities from indicators
        indicators = article.get('indicators', {})
        for indicator_type, values in indicators.items():
            for value in values:
                entity_id = f"{indicator_type}_{hash(value) % 100000}"
                
                # Entity
                entities_and_relations['entities'].append({
                    'id': entity_id,
                    'type': indicator_type,
                    'value': value
                })
                
                # Relation
                entities_and_relations['relations'].append({
                    'source': doc_id,
                    'target': entity_id,
                    'relation': 'mentions',
                    'type': indicator_type
                })
    
    # Remove duplicate entities
    seen_entities = set()
    unique_entities = []
    for entity in entities_and_relations['entities']:
        entity_key = (entity['type'], entity['value'])
        if entity_key not in seen_entities:
            seen_entities.add(entity_key)
            unique_entities.append(entity)
    entities_and_relations['entities'] = unique_entities
    
    kg_file = PROCESSED_DATA_DIR / f'knowledge_graph_data_{timestamp}.json'
    with open(kg_file, 'w', encoding='utf-8') as f:
        json.dump(entities_and_relations, f, indent=2, ensure_ascii=False)
    
    # 3. JSONL format (for streaming/batch processing)
    jsonl_file = PROCESSED_DATA_DIR / f'threat_intelligence_{timestamp}.jsonl'
    with open(jsonl_file, 'w', encoding='utf-8') as f:
        for article in articles:
            simplified_article = {
                'title': article['title'],
                'content': article['full_text'],
                'source': article['source'],
                'indicators': article.get('indicators', {}),
                'threat_score': article.get('threat_relevance_score', 0)
            }
            f.write(json.dumps(simplified_article, ensure_ascii=False) + '\n')
    
    # 4. Summary statistics
    export_summary = {
        'export_timestamp': datetime.now().isoformat(),
        'total_articles': len(articles),
        'sources': list(set(article['source'] for article in articles)),
        'total_entities': len(entities_and_relations['entities']),
        'total_relations': len(entities_and_relations['relations']),
        'files_created': {
            'llm_training': str(training_file),
            'knowledge_graph': str(kg_file),
            'jsonl_format': str(jsonl_file)
        },
        'statistics': {
            'avg_threat_score': sum(article.get('threat_relevance_score', 0) for article in articles) / len(articles),
            'total_technical_indicators': sum(sum(len(inds) for inds in article.get('indicators', {}).values()) for article in articles)
        }
    }
    
    summary_file = PROCESSED_DATA_DIR / f'export_summary_{timestamp}.json'
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(export_summary, f, indent=2, ensure_ascii=False)
    
    print("\\n✅ Export completed successfully!")
    print("="*60)
    print(f"📊 Exported {len(articles)} articles")
    print(f"🎯 {len(entities_and_relations['entities'])} unique entities")
    print(f"🔗 {len(entities_and_relations['relations'])} relations")
    print(f"📈 Avg threat score: {export_summary['statistics']['avg_threat_score']:.2f}")
    
    print("\\n📁 Files created:")
    for file_type, file_path in export_summary['files_created'].items():
        print(f"  {file_type}: {Path(file_path).name}")
    
    return export_summary

# Export the collected data
if 'collected_data' in locals() and collected_data:
    export_summary = export_for_llm_training(collected_data)
    print("\\n🎉 Data collection and export pipeline completed!")
else:
    print("⚠️  No data collected. Run the collection cell first.")


## Conclusion and Next Steps

This notebook implements a comprehensive threat intelligence data collection system following the LLM-TIKG methodology. The implementation includes:

### ✅ Key Components

1. **Robust Web Scraping Infrastructure**
   - Rate-limited, error-resilient scrapers with random delays
   - Platform-specific implementations for CISA, Fortinet, and Symantec
   - User agent rotation and retry mechanisms
   - Comprehensive error handling and logging

2. **Advanced Text Processing**
   - Paragraph structure preservation
   - Technical indicator extraction (IPs, domains, hashes, CVEs)
   - Threat relevance scoring
   - Content validation and quality checks

3. **Multiple Export Formats**
   - LLM training-ready JSON format
   - Knowledge graph entities and relations
   - JSONL for streaming processing
   - Statistical summaries and reports

### 🔍 Key Features

- **Quality Assurance**: Automated validation ensures only high-quality data is retained
- **Scalability**: Modular design allows easy addition of new sources
- **Reproducibility**: Comprehensive logging and configuration management
- **Error Handling**: Robust error recovery and fallback mechanisms
- **Rate Limiting**: Intelligent rate limiting with random delays to avoid blocking

### 🚀 Next Steps

1. **Expand Sources**:
   - Add more threat intelligence platforms
   - Implement RSS feed monitoring
   - Add support for API-based sources

2. **Enhance Processing**:
   - Improve entity extraction
   - Add relationship extraction
   - Implement cross-source validation

3. **Optimize Performance**:
   - Implement parallel processing
   - Add caching mechanisms
   - Optimize network requests

4. **Data Quality**:
   - Add more validation rules
   - Implement duplicate detection
   - Add content relevance scoring

This foundation provides the high-quality, structured threat intelligence data required for successful LLM-TIKG implementation.
